<a href="https://colab.research.google.com/github/r-dube/fakejobs/blob/main/fj_char_cnn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [24]:
# Load the modules used
import numpy as np
import pandas as pd
import re
import string
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix, roc_auc_score
from keras.models import Sequential
from keras.layers import Dense, Dropout, Embedding, LSTM, Bidirectional, GlobalMaxPool1D, Conv1D, MaxPool1D, Flatten
from keras.preprocessing.sequence import pad_sequences
from keras.initializers import Constant 
from keras.optimizers import Adam
from keras import metrics
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.text import Tokenizer
from collections import Counter

In [25]:
# NLTK to remove stopwords
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [26]:
# For reproducible results
import random as rn
import os
import tensorflow as tf
os.environ['PYTHONHASHSEED'] = '42'
os.environ['CUDA_VISIBLE_DEVICES'] = ''
np.random.seed(42)
rn.seed(42)
tf.random.set_seed(42)

In [27]:
# Set data_url, the location of the data
# Data is not loaded from a local file
# data_url="https://raw.githubusercontent.com/r-dube/fakejobs/main/data/fj_small.csv"
# data_url="https://raw.githubusercontent.com/r-dube/fakejobs/main/data/fj_medium.csv"
data_url="https://raw.githubusercontent.com/r-dube/fakejobs/main/data/fake_job_postings.csv"

In [28]:
def fj_load_df_from_url():
    """
    Load dataframe from csv file
    Input:
        None
    Returns:
        dataframe
    """

    df = pd.read_csv(data_url)

    print ('Loaded dataframe shape', df.shape)

    counts = fj_label_stats(df)
    print ('Not fraudulent', counts[0], 'Fraudulent', counts[1])

    print(df.describe())

    print ('NAs/NANs in data =>')
    print(df.isna().sum())

    return df

def fj_label_stats(df):
    """
    Very basic label statistics
    Input: 
        Dataframe
    Returns:
        Number of samples with 0, 1 as the label
    """
    counts = np.bincount(df['fraudulent'])
    return counts

def fj_txt_only(df):
    """
    Combine all the text fields, discard everything else except for the label
    Input: 
        Dataframe
    Returns:
        Processed dataframe
    """
    
    df.fillna(" ", inplace = True)

    df['text'] = df['title'] + ' ' + df['location'] + ' ' + df['department'] + \
    ' ' + df['company_profile'] + ' ' + df['description'] + ' ' + \
    df['requirements'] + ' ' + df['benefits'] + ' ' + df['employment_type'] + \
    ' ' + df['required_education'] + ' ' + df['industry'] + ' ' + df['function'] 

    del df['title']
    del df['location']
    del df['department']
    del df['company_profile']
    del df['description']
    del df['requirements']
    del df['benefits']
    del df['employment_type']
    del df['required_experience']
    del df['required_education']
    del df['industry']
    del df['function']  
    
    del df['salary_range']
    del df['job_id']
    del df['telecommuting']
    del df['has_company_logo']
    del df['has_questions']

    return df

In [29]:
df = fj_load_df_from_url()
df = fj_txt_only(df)
print('Maximum text length', df['text'].str.len().max())

Loaded dataframe shape (17880, 18)
Not fraudulent 17014 Fraudulent 866
             job_id  telecommuting  ...  has_questions    fraudulent
count  17880.000000   17880.000000  ...   17880.000000  17880.000000
mean    8940.500000       0.042897  ...       0.491723      0.048434
std     5161.655742       0.202631  ...       0.499945      0.214688
min        1.000000       0.000000  ...       0.000000      0.000000
25%     4470.750000       0.000000  ...       0.000000      0.000000
50%     8940.500000       0.000000  ...       0.000000      0.000000
75%    13410.250000       0.000000  ...       1.000000      0.000000
max    17880.000000       1.000000  ...       1.000000      1.000000

[8 rows x 5 columns]
NAs/NANs in data =>
job_id                     0
title                      0
location                 346
department             11547
salary_range           15012
company_profile         3308
description                1
requirements            2695
benefits                7210
telec

In [30]:
# Utilities to clean text

def remove_URL(text):
    url = re.compile(r"https?://\S+|www\.\S+")
    return url.sub(r"", text)

def remove_html(text):
    html = re.compile(r"<.*?>")
    return html.sub(r"", text)

def remove_emoji(string):
    emoji_pattern = re.compile(
        "["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
        u"\U00002702-\U000027B0"
        u"\U000024C2-\U0001F251"
        "]+",
        flags=re.UNICODE,
    )
    return emoji_pattern.sub(r"", string)

def remove_punct(text):
    table = str.maketrans("", "", string.punctuation)
    return text.translate(table)

In [31]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [32]:
stop = set(stopwords.words("english"))

def remove_stopwords(text):
    text = [word.lower() for word in text.split() if word.lower() not in stop]

    return " ".join(text)

In [33]:
# clean text
# commenting out some preprocessing as charcnn does not need it
# df['text'] = df['text'].map(lambda x: remove_URL(x))
# df['text'] = df['text'].map(lambda x: remove_html(x))
df['text'] = df['text'].map(lambda x: remove_emoji(x))
# df['text'] = df['text'].map(lambda x: remove_punct(x))
df['text'] = df["text"].map(remove_stopwords)

In [34]:
# Count unique words
def counter_word(text):
    count = Counter()
    for i in text.values:
        for word in i.split():
            count[word] += 1
    return count

text = df['text']

counter = counter_word(text)
num_words = len(counter)
print ("Number of words: ", num_words)

# Max number of words in a sequence
max_length = 250

Number of words:  230664


In [35]:
# train-test split
train_text, test_text, train_labels , test_labels = train_test_split(df['text'], df['fraudulent'] , test_size = 0.15)
print(train_text.shape)

(15198,)


In [36]:
alphabet="abcdefghijklmnopqrstuvwxyz0123456789-,;.!?:'\"/\\|_@#$%^&*~`+-=<>()[]{}"
embedding_size = len(alphabet)
input_size=1014
num_of_classes=2
dict = {}  # Maps each character to an integer
for idx, char in enumerate(alphabet):
  dict[char] = idx + 1

print(embedding_size)
print (dict)

69
{'a': 1, 'b': 2, 'c': 3, 'd': 4, 'e': 5, 'f': 6, 'g': 7, 'h': 8, 'i': 9, 'j': 10, 'k': 11, 'l': 12, 'm': 13, 'n': 14, 'o': 15, 'p': 16, 'q': 17, 'r': 18, 's': 19, 't': 20, 'u': 21, 'v': 22, 'w': 23, 'x': 24, 'y': 25, 'z': 26, '0': 27, '1': 28, '2': 29, '3': 30, '4': 31, '5': 32, '6': 33, '7': 34, '8': 35, '9': 36, '-': 60, ',': 38, ';': 39, '.': 40, '!': 41, '?': 42, ':': 43, "'": 44, '"': 45, '/': 46, '\\': 47, '|': 48, '_': 49, '@': 50, '#': 51, '$': 52, '%': 53, '^': 54, '&': 55, '*': 56, '~': 57, '`': 58, '+': 59, '=': 61, '<': 62, '>': 63, '(': 64, ')': 65, '[': 66, ']': 67, '{': 68, '}': 69}


In [37]:
# return one-hot-vector from string
def str_to_ohv(s, ohv):
  max_length = min(len(s), input_size)
  for i in range(0, max_length):
    c = s[i]
    if c in dict:
      ohv[i, dict[c]-1] = 1
  return ohv

In [38]:
"""
# test for the character-by-character encoding
t = np.zeros((input_size, embedding_size))
print (str_to_ohv("ab}", t))
"""

'\n# test for the character-by-character encoding\nt = np.zeros((input_size, embedding_size))\nprint (str_to_ohv("ab}", t))\n'

In [39]:
num_jobs = train_text.shape[0]
train_t = np.zeros((num_jobs, input_size, embedding_size), dtype=np.int8)
i = 0
for _, val in train_text.iteritems():
  str_to_ohv(val, train_t[i])
  i=i+1

In [40]:
model = Sequential()
model.add(Conv1D(32, kernel_size=3, padding="valid", activation="relu", strides=1, input_shape=(input_size, embedding_size)))
model.add(MaxPool1D(pool_size=3, padding="valid"))
model.add(Conv1D(32, kernel_size=3, padding="valid", activation="relu", strides=1, input_shape=(input_size, embedding_size)))
model.add(MaxPool1D(pool_size=3, padding="valid"))
model.add(Flatten())
model.add(Dense(128, activation="relu"))
model.add(Dropout(0.5))
model.add(Dense(128, activation="relu"))
model.add(Dropout(0.5))
model.add(Dense(1,activation="sigmoid"))
model.compile(optimizer = 'adam' , loss = 'binary_crossentropy' , metrics = ['accuracy', tf.keras.metrics.FalsePositives(), tf.keras.metrics.FalseNegatives()])
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_2 (Conv1D)            (None, 1012, 32)          6656      
_________________________________________________________________
max_pooling1d_2 (MaxPooling1 (None, 337, 32)           0         
_________________________________________________________________
conv1d_3 (Conv1D)            (None, 335, 32)           3104      
_________________________________________________________________
max_pooling1d_3 (MaxPooling1 (None, 111, 32)           0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 3552)              0         
_________________________________________________________________
dense_3 (Dense)              (None, 128)               454784    
_________________________________________________________________
dropout_2 (Dropout)          (None, 128)              

In [41]:
model.fit(train_t, train_labels, epochs = 10)

Epoch 1/10
475/475 [==============================] - 38s 78ms/step - loss: 0.2313 - accuracy: 0.9436 - false_positives_1: 13.9727 - false_negatives_1: 367.5651
Epoch 2/10
475/475 [==============================] - 36s 76ms/step - loss: 0.1360 - accuracy: 0.9531 - false_positives_1: 1.2143 - false_negatives_1: 353.7878
Epoch 3/10
475/475 [==============================] - 37s 78ms/step - loss: 0.0686 - accuracy: 0.9783 - false_positives_1: 22.5609 - false_negatives_1: 146.3571
Epoch 4/10
475/475 [==============================] - 37s 78ms/step - loss: 0.0390 - accuracy: 0.9868 - false_positives_1: 22.5420 - false_negatives_1: 78.9055
Epoch 5/10
475/475 [==============================] - 37s 78ms/step - loss: 0.0241 - accuracy: 0.9920 - false_positives_1: 18.3971 - false_negatives_1: 43.9601
Epoch 6/10
475/475 [==============================] - 37s 78ms/step - loss: 0.0153 - accuracy: 0.9941 - false_positives_1: 15.9580 - false_negatives_1: 27.8403
Epoch 7/10
475/475 [==================

In [42]:
num_jobs = test_text.shape[0]
test_t = np.zeros((num_jobs, input_size, embedding_size), dtype=np.int8)
i = 0
for _, val in test_text.iteritems():
  str_to_ohv(val, test_t[i])
  i=i+1

In [43]:
pred_soft = model.predict(test_t)

In [44]:
# pred = np.around(pred_soft, decimals = 0)
pred = np.where(pred_soft > 0.15, 1, 0)

acc = accuracy_score(pred, test_labels)
f1 = f1_score(pred, test_labels)

cm = confusion_matrix(test_labels, pred)
tn = cm[0][0]
fn = cm[1][0]
tp = cm[1][1]
fp = cm[0][1]

print('Accuracy score: {:.4f}'.format(acc), 'F1 score: {:.4f}'.format(f1))
print('False Positives: {:.0f}'.format(fp), 'False Negatives: {:.0f}'.format(fn))
print('Confusion matrix:\n', cm)

Accuracy score: 0.9739 F1 score: 0.7083
False Positives: 11 False Negatives: 59
Confusion matrix:
 [[2527   11]
 [  59   85]]


In [45]:
auc = roc_auc_score(test_labels, pred_soft)
print('AUC score: {:.4f}'.format(auc))

AUC score: 0.9448


In [46]:
""""
# Uncomment to save results on drive to a csv file
df_results1 = pd.DataFrame(data=test_labels)
df_results1.reset_index(drop=True, inplace=True)
df_results2 = pd.DataFrame(data=pred_soft, columns=['char_cnn'])
df_results = pd.concat([df_results1, df_results2], axis=1)

from google.colab import drive
drive.mount('/content/drive')
results_file='/content/drive/My Drive/Results/fj_char_cnn.csv'

df_results.to_csv(results_file)
"""

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
